In [1]:
import pandas as pd

In [3]:
data=pd.read_excel("/content/Japan_Convenience_Store_Products_with_Volume.xlsx")
data

,Product Name,Company,Height (cm),Width (cm),Length (cm),Weight (g),Volume (cm³)
0,Brand08_250ml_Noodles,Brand08,12.8,11.4,12.8,176.3,1867.776
1,Brand02_1L_Soda,Brand02,9.9,10.9,15.4,1369.1,1661.814
2,Brand48_250ml_Soda,Brand48,11.1,3.2,4.7,1282.5,166.944
3,Brand22_750ml_Bread,Brand22,10.1,14.7,13.4,521.5,1989.498
4,Brand37_250ml_Coffee,Brand37,16.6,4.7,13.6,122.8,1061.072
...,...,...,...,...,...,...,...
95,Brand17_1L_Tea,Brand17,15.4,10.8,7.4,646.8,1230.768
96,Brand48_500ml_Sandwich,Brand48,8.7,9.5,17.7,1165.1,1462.905
97,Brand14_1L_Cake,Brand14,11.4,3.0,9.6,988.2,328.320
98,Brand01_100ml_Water,Brand01,20.7,7.2,12.5,1062.2,1863.000


In [5]:
df=data

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product Name  100 non-null    object 
 1   Company       100 non-null    object 
 2   Height (cm)   100 non-null    float64
 3   Width (cm)    100 non-null    float64
 4   Length (cm)   100 non-null    float64
 5   Weight (g)    100 non-null    float64
 6   Volume (cm³)  100 non-null    float64
dtypes: float64(5), object(2)
memory usage: 5.6+ KB


In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Height (cm),100.0,16.97300,7.489304,5.00,9.97500,16.6000,24.125,29.700
Width (cm),100.0,8.45400,4.121131,2.00,5.20000,8.2500,11.875,15.000
Length (cm),100.0,11.90900,4.943051,2.10,9.20000,12.6000,15.800,19.900
Weight (g),100.0,775.10000,423.838440,67.10,429.25000,701.8500,1133.150,1498.500
Volume (cm³),100.0,1801.18659,1488.719616,76.23,567.67625,1463.7885,2785.809,7369.512


In [12]:
# Create a DataFrame for the convenience store bag sizes
bag_data = {
    "Size": ["Small (S)", "Medium (M)", "Large (L)"],
    "Approx. Dimensions (cm)": ["30 × 20 × 10", "40 × 25 × 15", "50 × 30 × 20"],
    "Volume Capacity (liters)": ["5", "10–12", "20–25"],
    "Lower Weight Capacity (kg)": ["2", "4", "6"],
    "Higher Weight Capacity (kg)": ["3", "5", "7"],
    "Typical Use Cases": [
        "1–3 items (e.g., drink, onigiri, snack)",
        "Lunch boxes, drinks, 5–6 items",
        "Full meal sets, multiple drinks/snacks, household goods"
    ]
}

bag_df = pd.DataFrame(bag_data)
bag_df


,Size,Approx. Dimensions (cm),Volume Capacity (liters),Lower Weight Capacity (kg),Higher Weight Capacity (kg),Typical Use Cases
0,Small (S),30 × 20 × 10,5,2,3,"1–3 items (e.g., drink, onigiri, snack)"
1,Medium (M),40 × 25 × 15,10–12,4,5,"Lunch boxes, drinks, 5–6 items"
2,Large (L),50 × 30 × 20,20–25,6,7,"Full meal sets, multiple drinks/snacks, househ..."


In [24]:
def suggest_bags(total_weight, total_volume):
    # Define bag sizes with their capacities
    bags = {
        "Small (S)": {"vol_capacity": 5000,  "weight_capacity": 3000,  "dimensions": "30 × 20 × 10 cm"},
        "Medium (M)": {"vol_capacity": 12000, "weight_capacity": 5000,  "dimensions": "40 × 25 × 15 cm"},
        "Large (L)": {"vol_capacity": 25000, "weight_capacity": 7000, "dimensions": "50 × 30 × 20 cm"}
    }

    # Convert from cm³ to liters for display
    total_volume_liters = total_volume / 1000
    total_weight_kg = total_weight / 1000

    # Check if everything fits in a single bag
    recommendation = ""
    explanation = ""

    # Try to fit in a single bag first (from smallest to largest)
    for size, capacity in bags.items():
        if (total_volume <= capacity["vol_capacity"] and
            total_weight <= capacity["weight_capacity"]):
            recommendation = f"1 {size} bag"
            explanation = (f"A single {size} bag is sufficient.\n"
                          f"Your items: {total_weight:.1f}g ({total_weight_kg:.2f}kg) / "
                          f"{total_volume:.1f}cm³ ({total_volume_liters:.2f}L)\n"
                          f"Bag capacity: {capacity['weight_capacity']}g / {capacity['vol_capacity']/1000}L")
            return recommendation, explanation

    # Start with checking if one large bag works
    if (total_volume <= bags["Large (L)"]["vol_capacity"] and
        total_weight <= bags["Large (L)"]["weight_capacity"]):
        return "1 Large (L) bag", (f"A single Large bag is sufficient.\n"
                              f"Your items: {total_weight:.1f}g ({total_weight_kg:.2f}kg) / "
                              f"{total_volume:.1f}cm³ ({total_volume_liters:.2f}L)")

    # Try combinations - starting with the largest bag to minimize total number
    remaining_volume = total_volume
    remaining_weight = total_weight
    bag_count = {"Large (L)": 0, "Medium (M)": 0, "Small (S)": 0}

    # First add as many large bags as needed
    while (remaining_volume > 0 or remaining_weight > 0):
        if (remaining_volume <= bags["Large (L)"]["vol_capacity"] and
            remaining_weight <= bags["Large (L)"]["weight_capacity"]):
            # Can fit the rest in one large bag
            bag_count["Large (L)"] += 1
            remaining_volume = 0
            remaining_weight = 0
        elif (remaining_volume <= bags["Medium (M)"]["vol_capacity"] and
              remaining_weight <= bags["Medium (M)"]["weight_capacity"]):
            # Can fit the rest in one medium bag
            bag_count["Medium (M)"] += 1
            remaining_volume = 0
            remaining_weight = 0
        elif (remaining_volume <= bags["Small (S)"]["vol_capacity"] and
              remaining_weight <= bags["Small (S)"]["weight_capacity"]):
            # Can fit the rest in one small bag
            bag_count["Small (S)"] += 1
            remaining_volume = 0
            remaining_weight = 0
        else:
            # Need to use the largest bag and continue
            bag_count["Large (L)"] += 1
            remaining_volume -= bags["Large (L)"]["vol_capacity"]
            remaining_weight -= bags["Large (L)"]["weight_capacity"]
            if remaining_volume < 0: remaining_volume = 0
            if remaining_weight < 0: remaining_weight = 0

    # Create recommendation string
    recommendation_parts = []
    for size, count in bag_count.items():
        if count > 0:
            recommendation_parts.append(f"{count} {size} bag{'s' if count > 1 else ''}")

    recommendation = " and ".join(recommendation_parts)
    explanation = (f"Multiple bags needed for {total_weight:.1f}g ({total_weight_kg:.2f}kg) / "
                  f"{total_volume:.1f}cm³ ({total_volume_liters:.2f}L)\n"
                  f"Recommended combination: {recommendation}")

    return recommendation, explanation

def sum_product_metrics_with_bag_suggestion(data):

    # Show available products in the dataset
    print("Available products in the dataset:")
    for i, product in enumerate(data['Product Name']):
        print(f"  {product}")
    print("\nEnter product names one by one. Type 'end' when finished.")

    # Collect product names
    product_names = []
    while True:
        product_input = input("Enter product name (or 'end' to finish): ")
        if product_input.lower() == 'end':
            break
        product_names.append(product_input)

    if not product_names:
        print("No products were specified.")
        return 0, 0, ""

    # Filter the data for the specified products
    filtered_data = data[data['Product Name'].isin(product_names)]

    # Check for products not found
    found_products = filtered_data['Product Name'].tolist()
    not_found = [p for p in product_names if p not in found_products]
    if not_found:
        print(f"Warning: The following products were not found: {', '.join(not_found)}")

    # Check if any products were found
    if filtered_data.empty:
        print("No matching products found in the dataset.")
        return 0, 0, ""

    # Calculate the sums
    total_weight = filtered_data['Weight (g)'].sum()
    total_volume = filtered_data['Volume (cm³)'].sum()

    # Display the results
    print("\nResults:")
    print(f"Total Weight for selected products: {total_weight:.1f} g")
    print(f"Total Volume for selected products: {total_volume:.1f} cm³")

    # Get bag suggestion
    bag_recommendation, explanation = suggest_bags(total_weight, total_volume)
    print("\nBag Recommendation:")
    print(explanation)

    # Show selected products with their individual weight and volume
    print("\nSelected Products:")
    print("Product Name | Weight (g) | Volume (cm³)")
    print("-" * 50)
    for _, row in filtered_data.iterrows():
        print(f"{row['Product Name']} | {row['Weight (g)']:.1f} g | {row['Volume (cm³)']:.1f} cm³")

    return total_weight, total_volume, bag_recommendation

# To use the function:
sum_product_metrics_with_bag_suggestion(data)

Available products in the dataset:
  Brand08_250ml_Noodles
  Brand02_1L_Soda
  Brand48_250ml_Soda
  Brand22_750ml_Bread
  Brand37_250ml_Coffee
  Brand10_100ml_Bread
  Brand13_100ml_Juice
  Brand31_500ml_Water
  Brand44_750ml_Water
  Brand47_250ml_ChewingGum
  Brand01_750ml_Chocolate
  Brand06_750ml_Coffee
  Brand14_750ml_Cigarette
  Brand45_250ml_Chocolate
  Brand38_250ml_Ramen
  Brand14_1L_Milk
  Brand28_500ml_Cigarette
  Brand44_500ml_Cigarette
  Brand24_750ml_Chocolate
  Brand34_250ml_Bread
  Brand10_1L_Alcohol
  Brand33_1L_IceCream
  Brand43_500ml_Ramen
  Brand17_250ml_ChewingGum
  Brand08_500ml_Ramen
  Brand03_1L_Cigarette
  Brand45_500ml_Ramen
  Brand23_500ml_Yogurt
  Brand18_1L_IceCream
  Brand02_100ml_Bread
  Brand17_1L_Milk
  Brand16_1L_Cigarette
  Brand33_750ml_Soda
  Brand41_250ml_Chips
  Brand02_100ml_Soda
  Brand02_1L_Yogurt
  Brand29_750ml_Tea
  Brand23_1L_Ramen
  Brand17_500ml_Sandwich
  Brand47_750ml_Chips
  Brand46_500ml_Noodles
  Brand11_1L_Chocolate
  Brand50_750ml_C

(np.float64(4299.9), np.float64(6848.964999999999), '1 Medium (M) bag')